# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [8]:
import pandas as pd

In [9]:
ratings = pd.read_csv('ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [10]:
count_r = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': ['min', 'max']})

In [11]:
count_r.columns = ['rating', 'start', 'stop'] #к агрегированным колонкам надо как-то обращаться
count_r

,rating,start,stop
userId,,,
1,20,1260759108,1260759205
2,76,835355395,835356246
3,51,1298861589,1298932787
4,204,949778714,949982274
5,100,1163373044,1163375145
...,...,...,...
667,68,847271220,847272234
668,20,993613196,993613478
669,37,1015829081,1015829766


In [12]:
old_users = count_r[count_r['rating'] >= 100].copy()
old_users['ltv'] = old_users.apply(lambda x: x.stop - x.start, axis = 1) 

In [13]:
old_users['ltv']

userId
4         203560
5           2101
8          85187
15     471393496
17          8053
         ...    
656         3053
659     31609411
664     98180439
665     54131471
671     11283984
Name: ltv, Length: 263, dtype: int64

# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [14]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [15]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [16]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [17]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [18]:
base1 = pd.DataFrame(client_base['client_id']).merge(air, how = 'left').merge(auto, how = 'left').merge(rzd, how = 'left')
base1

,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,NaN,NaN,1093.0
1,112,NaN,NaN,2810.0
2,113,NaN,57483.0,10283.0
3,114,NaN,83.0,5774.0
4,115,81.0,912.0,981.0
5,116,4.0,4834.0,NaN
6,117,13.0,98.0,NaN
7,118,173.0,NaN,NaN


In [19]:
base2 = client_base.merge(air, how = 'left').merge(auto, how = 'left').merge(rzd, how = 'left')
base2

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,NaN,57483.0,10283.0
3,114,Мира 14,NaN,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,NaN
6,117,Панфиловская 33,13.0,98.0,NaN
7,118,Мастеркова 4,173.0,NaN,NaN


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Задача на писанину? Ок.

Сложно вот так рассуждать, не зная в чем наш бизнес, но:

1. Мы наверняка знаем время получения координат, и также время визита/покупки. Берем ближайщее местоположение к визиту/покупки по времени.

2. Если разлет между набором координат, которые мы получили во временном интвервале, в котором была осуществлена покупка, то мы можем апроксимировать координаты вычислив разницу между ними, деленную на количество секунд между этими координатами, и умноженную на количество секунд, которое прошло от первой координаты до визита/покупки.

3. Ес-но мы можем получить страну, город, регион и все в таком духе из координат. Из координат мы можем понять насколько динамично передвигается человек: сидит ли он дома, часто ли гуляет, ездит на машине или общественном транспорте, когда летает на самолете/ступе. 